In [12]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate



In [16]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="Bob", user_input="What is your name?")

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

In [17]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


In [29]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./bella_vista.txt")
docs = loader.load()

In [40]:
docs[0]

Document(metadata={'source': './bella_vista.txt'}, page_content="Q: What are the hours of operation for Bella Vista?\nA: Bella Vista is open from 11 a.m. to 11 p.m. from Monday to Saturday. On Sundays, we welcome guests from 12 p.m. to 10 p.m.\n\nQ: What type of cuisine does Bella Vista serve?\nA: Bella Vista offers a delightful blend of Mediterranean and contemporary American cuisine. We pride ourselves on using the freshest ingredients, many of which are sourced locally.\n\nQ: Do you offer vegetarian or vegan options at Bella Vista?\nA: Absolutely! Bella Vista boasts a diverse menu that includes a variety of vegetarian and vegan dishes. Our chefs are also happy to customize dishes based on dietary needs.\n\nQ: Is Bella Vista family-friendly?\nA: Yes, Bella Vista is a family-friendly establishment. We have a dedicated kids' menu and offer high chairs and booster seats for our younger guests.\n\nQ: Can I book private events at Bella Vista?\nA: Certainly! Bella Vista has a private dinin

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents(docs)

In [44]:
from langchain_core.documents import Document

test = "In this podcast, the speakers discuss the pressing issue of the environment and the need for immediate attention. Speaker_1 emphasizes the importance of reducing carbon emissions and investing in renewable energy. Speaker_2 adds that while renewable energy is crucial, we must also prioritize conserving natural habitats and protecting wildlife from the effects of climate change. Speaker_1 agrees with both points but highlights the importance of considering the economic impact of environmental policies. They stress the need for solutions that balance ecological preservation with economic growth. Overall, the speakers advocate for a holistic approach to addressing environmental challenges."
document = Document(
    page_content= test,
    metadata={"source": "audio.wav"}
)

document


Document(metadata={'source': 'audio.wav'}, page_content='In this podcast, the speakers discuss the pressing issue of the environment and the need for immediate attention. Speaker_1 emphasizes the importance of reducing carbon emissions and investing in renewable energy. Speaker_2 adds that while renewable energy is crucial, we must also prioritize conserving natural habitats and protecting wildlife from the effects of climate change. Speaker_1 agrees with both points but highlights the importance of considering the economic impact of environmental policies. They stress the need for solutions that balance ecological preservation with economic growth. Overall, the speakers advocate for a holistic approach to addressing environmental challenges.')

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
documents = text_splitter.split_documents([document])
documents



[Document(metadata={'source': 'audio.wav'}, page_content='In this podcast, the speakers discuss the pressing issue of the environment and the need for'),
 Document(metadata={'source': 'audio.wav'}, page_content='and the need for immediate attention. Speaker_1 emphasizes the importance of reducing carbon'),
 Document(metadata={'source': 'audio.wav'}, page_content='of reducing carbon emissions and investing in renewable energy. Speaker_2 adds that while renewable'),
 Document(metadata={'source': 'audio.wav'}, page_content='while renewable energy is crucial, we must also prioritize conserving natural habitats and'),
 Document(metadata={'source': 'audio.wav'}, page_content='habitats and protecting wildlife from the effects of climate change. Speaker_1 agrees with both'),
 Document(metadata={'source': 'audio.wav'}, page_content='agrees with both points but highlights the importance of considering the economic impact of'),
 Document(metadata={'source': 'audio.wav'}, page_content='economic im

In [54]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()


db=FAISS.from_documents(documents,OpenAIEmbeddings())


db

In [53]:
query="renewable energy is important "
result=db.similarity_search(query)
result

[Document(metadata={'source': 'audio.wav'}, page_content='of reducing carbon emissions and investing in renewable energy. Speaker_2 adds that while renewable'),
 Document(metadata={'source': 'audio.wav'}, page_content='while renewable energy is crucial, we must also prioritize conserving natural habitats and'),
 Document(metadata={'source': 'audio.wav'}, page_content='and the need for immediate attention. Speaker_1 emphasizes the importance of reducing carbon'),
 Document(metadata={'source': 'audio.wav'}, page_content='advocate for a holistic approach to addressing environmental challenges.')]

In [47]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

prompt

ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \nI will tip you $1000 if the user finds the answer helpful. \n<context>\n{context}\n</context>\nQuestion: {input}'))])

In [57]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)


In [58]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10edbee30>)

In [59]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [60]:
response=retrieval_chain.invoke({"input":"why  is renewable energy important ? "})

In [77]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data.txt")
docs = loader.load()
text = docs[0].page_content

In [71]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

prompt

ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \nI will tip you $1000 if the user finds the answer helpful. \n<context>\n{context}\n</context>\nQuestion: {input}'))])

In [91]:
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI()
def give_documents(text): 
    document = Document(
        page_content= text,
        metadata={"source": "audio.wav"}
    ) 
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
    documents = text_splitter.split_documents([document])
    return documents 

def give_retriver(documents): 
    db=FAISS.from_documents(documents,OpenAIEmbeddings())
    return db.as_retriever()
     

def give_chain(retriever): 
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | format_docs, "input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain 

documents = give_documents(text)
retriever = give_retriver(documents)
#retriever.invoke("what are threats of AI ? ")

chain = give_chain(retriever)
chain.invoke("can you summarize this video")

'Based on the provided context, the video is about sharing tips and advice on becoming a successful YouTuber, including video editing, filmmaking, and earning money through content creation. The speaker also mentions the importance of hard work and offers guidance on choosing the right topic for content creation.'

In [89]:
print(chain.invoke("Explain me neet exam and marking system"))

Based on the provided context, it seems that the user is studying for an edition of the NEET exam and discussing the grace marks given by NTA (National Testing Agency) for discrepancies in the exam. The NEET exam is a medical entrance exam that consists of multiple-choice questions (MCQs) from subjects like Physics, Chemistry, and Zoology. In this particular case, there were discrepancies in the answers provided, leading to grace marks being awarded to 6 students for time-related issues.

The NTA used a marking system that involved giving grace marks to students who may have been affected by these discrepancies. The exact number of grace marks awarded in this particular instance is not specified in the context, but it was enough to help these students in their exam scores. The grace marks were likely given to ensure fairness and to compensate for any errors or issues in the exam.

In summary, the NEET exam is a competitive medical entrance exam with MCQs from various subjects, and the 

In [100]:
from langchain_core.prompts import ChatPromptTemplate
system_template =  """
Given the following context translate this to english if required and 
answer the user question based on context. keep answers long and include every minute details .Try to answer pointwise 
{context}
"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{question}")]
)
result = prompt_template.invoke({"context": "some context", "question": "hi"})

print(result)


messages=[SystemMessage(content='\nGiven the following context translate this to english if required and \nanswer the user question based on context. keep answers long and include every minute details .Try to answer pointwise \nsome context\n'), HumanMessage(content='hi')]


"SPEAKER_00: अरंदर मोटी जी Deadline थी 9th March रेजिस्टेर करने ही जोकि नॉर्मल है एक महीने तक का समय दिया गया लेकिन 9th March को एक हाफ़ते के लिए और एक्स्टेंड कर दिया जाता है इस Deadline में अब ये भी कोई इतनी बड़ी बात नहीं है क्योंकि हो सकता है कुछ Students इतने मगण हो पढ़ाई करने में कि वो इस एक महीने में रेजिस्टेर करना ही भूल गई एक्साम के लिए ठीक है उनके लिए एक हाफ़ते और एक्स्टेंड कर दी Deadline लेकिन एक महीने बाद और 9th April को फिर से इस registration link को खोला जाता है 9th April से लेकर 10th April तक stakeholders request पर और इसके बाद एक और बार एक correction window दी जाती है 11th April से लेकर 15th April तक ये किस stakeholder की request थी इसके बाद exam conduct किया जाता है 5th May को और इसके बाद भी धेर सारी paper leaks की ख़बरे आती हैं अलग-अलग states से कभी गुजराद से, कभी बिहार से, तो कभी ओरिस्सा से येकिन असली मज़ाग शुरू होता है 4th June को जब results declare होते हैं NEET का exam एक MCQ exam है जिसमें 4 अलग-अलग subjects में सवाल पूछे जाते हैं Physics, Chemistry, Zoology और Bot

In [101]:


chain = prompt_template | llm | StrOutputParser()

result_chain = chain.invoke({"context": text , "question":"summarize this whole content"})

In [102]:
print(result_chain)

The speaker discusses the NEET exam process, including registration deadlines, extensions, and the conduct of the exam. They mention the National Testing Agency (NTA) and its establishment to bring transparency to exams. The speaker highlights issues with the NEET exam, including paper leaks, discrepancies in scoring, and the controversy surrounding grace marks given to certain students. They also mention the petition signed by over 40,000 students to reevaluate the exam results. Additionally, the speaker criticizes the Education Minister for focusing more on PR activities than addressing education system issues. The speaker emphasizes the need for accountability and transparency in the education system. Overall, the content points to systemic flaws and irregularities in the NEET exam process, raising concerns about the future of students and the integrity of the education system.
